In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [ ]:
// Fork-Join network example 6
val model = Network("model")

In [ ]:
// Create network
val source = Source(model, "Source")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
val queue2 = Queue(model, "Queue2", SchedStrategy.FCFS)
val fork1 = Fork(model, "Fork1")
val join1 = Join(model, "Join1", fork1)
val queue3 = Queue(model, "Queue3", SchedStrategy.FCFS)
val queue4 = Queue(model, "Queue4", SchedStrategy.FCFS)
val fork2 = Fork(model, "Fork2")
val join2 = Join(model, "Join2", fork2)
val sink = Sink(model, "Sink")

val jobclass1 = OpenClass(model, "class1")

In [ ]:
// Service configurations
source.setArrival(jobclass1, Exp(0.4))
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(1.0))
queue3.setService(jobclass1, Exp(1.0))
queue4.setService(jobclass1, Exp(1.0))

In [ ]:
// Two sequential fork-join stages routing
val P = model.initRoutingMatrix()
P.set(jobclass1, jobclass1, source, fork1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join1, 1.0)
P.set(jobclass1, jobclass1, queue2, join1, 1.0)
P.set(jobclass1, jobclass1, join1, fork2, 1.0)
P.set(jobclass1, jobclass1, fork2, queue3, 1.0)
P.set(jobclass1, jobclass1, fork2, queue4, 1.0)
P.set(jobclass1, jobclass1, queue3, join2, 1.0)
P.set(jobclass1, jobclass1, queue4, join2, 1.0)
P.set(jobclass1, jobclass1, join2, sink, 1.0)
model.link(P)

In [ ]:
// Solve with multiple methods
val solvers = mutableListOf<NetworkSolver>()

solvers.add(JMT(model, "seed", 23000))
solvers.add(MVA(model))

for ((i, solver) in solvers.withIndex()) {
    println("SOLVER ${i+1}: ${solver.name.replace("Solver", "")}")
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error: ${e.message}")
    }
}

This open series fork-join model demonstrates:

1. **Open Pipeline Architecture**:
   - Source → Stage1 → Stage2 → Sink
   - Jobs flow through system once and exit
   - No recirculation or feedback loops

2. **Sequential Fork-Join Processing**:
   - **Stage 1**: Fork1 → {Queue1, Queue2} → Join1
   - **Stage 2**: Fork2 → {Queue3, Queue4} → Join2
   - Each job must complete both stages

3. **Uniform Service Configuration**:
   - All queues have rate 1.0 (identical processing)
   - Source generates jobs at rate 0.4
   - FCFS scheduling maintains job order

4. **Load Distribution**:
   - **Arrival rate**: 0.4 jobs/time unit
   - **Effective load per queue**: 0.4 × 50% = 0.2 utilization
   - Well below saturation (utilization < 1.0)

5. **Response Time Composition**:
   - Each job experiences 4 queue delays
   - 2 synchronization delays (at joins)
   - Total response time is sum of all components

6. **Throughput Analysis**:
   - **System throughput**: Limited by source rate (0.4)
   - **Stage capacity**: Each stage can handle much higher load
   - **Bottleneck**: Source arrival rate, not processing capacity

The model represents:
- **Manufacturing assembly lines**: Multi-stage production with parallel operations
- **Data processing pipelines**: Sequential transformations with parallel computation
- **Software workflows**: Multi-phase processing systems
- **Document processing**: Multi-step validation and transformation

Key insights:
- **Under-utilization**: Low arrival rate keeps all queues well below capacity
- **Synchronization overhead**: Join delays contribute significantly to response time
- **Scalability potential**: System can handle much higher loads

**Performance Pattern**: In this configuration, the system is arrival-limited rather than capacity-limited, with ample processing resources available. Response times are dominated by the number of processing stages rather than queueing delays.

**Contrast with Closed System**: Unlike the closed version, this open system doesn't build up job populations at bottlenecks, resulting in more predictable and linear performance characteristics.